In [3]:
# Part I: Based on Feature Selection, Cleaning, and Preprocessing to Construct an Input from Data Source
# (a) Examine the values of each attribute and Select a set of attributes only that would affect to predict
#     future bike buyers to create your input for data mining algorithms. Remove all the unnecessary
#     attributes. (Select features just by analysis).
# (b) Create a new Data Frame with the selected attributes only.
# (c) Determine a Data value type (Discrete, or Continuous, then Nominal, Ordinal, Interval, Ratio) of
#     each attribute in your selection to identify preprocessing tasks to create input for your data mining.

# Part II: Data Preprocessing and Transformation
# Depending on the data type of each attribute, transform each object from your preprocessed data.
# Use all the data rows (~18000 rows) with the selected features as input to apply all the tasks below, do
# not perform each task on the smaller data set that you got from your random sampling result.
# (a) Handling Null values
# (b) Normalization
# (c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different
#     values
# (d) Standardization/Normalization
# (e) Binarization (One Hot Encoding)

# Part III: Calculating Proximity/Correlation Analysis of two features
# Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each
# nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct
# similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively.
# (a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two
#     following objects of your transformed input data.
# (b) Calculate Correlation between two features Commute Distance and Yearly Income

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# --- Load Datasets ---
# Assuming the CSV files are uploaded and accessible in the Colab environment.
# Replace with the actual file paths if they are different.
try:
    customers_df = pd.read_csv('AWCustomers.csv')
    sales_df = pd.read_csv('AWSales.csv')
    # Test files are not used in this assignment based on the prompt's scope,
    # but are included here for completeness if needed for future tasks.
    # test_classification_df = pd.read_csv('AWTest-Classification.csv')
    # test_regression_df = pd.read_csv('AWTest-Regression.csv')
except FileNotFoundError:
    print("Ensure 'AWCustomers.csv' and 'AWSales.csv' are uploaded to your Colab environment.")
    exit()

# --- Part I: Feature Selection, Cleaning, and Preprocessing Input Construction ---

# Merge the two dataframes on CustomerID
df_merged = pd.merge(customers_df, sales_df, on='CustomerID', how='inner')

# (a) & (b) Select relevant attributes and create a new DataFrame
# Attributes selected for predicting bike buyers:
# - BirthDate (will be converted to Age)
# - Education (Ordinal)
# - Occupation (Nominal)
# - Gender (Nominal)
# - MaritalStatus (Nominal)
# - HomeOwnerFlag (Binary/Discrete)
# - NumberCarsOwned (Discrete/Ratio)
# - NumberChildrenAtHome (Discrete/Ratio)
# - TotalChildren (Discrete/Ratio)
# - YearlyIncome (Continuous/Ratio)
# - CountryRegionName (Nominal - broader geographical indicator)
# - AvgMonthSpend (Continuous/Ratio - historical spend)
# - BikeBuyer (Target variable - Binary)

selected_features = [
    'BirthDate', 'Education', 'Occupation', 'Gender', 'MaritalStatus',
    'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
    'TotalChildren', 'YearlyIncome', 'CountryRegionName', 'AvgMonthSpend', 'BikeBuyer'
]

df_selected = df_merged[selected_features].copy()

# (c) Determine Data Value Type of each attribute
# This is described in comments for each feature during preprocessing steps.

# --- Part II: Data Preprocessing and Transformation ---

# 1. Convert BirthDate to Age
df_selected['BirthDate'] = pd.to_datetime(df_selected['BirthDate'])
current_year = pd.to_datetime('2017-03-06').year # Using a reference date from LastUpdated column
df_selected['Age'] = current_year - df_selected['BirthDate'].dt.year
df_selected.drop('BirthDate', axis=1, inplace=True)

# 2. (a) Handling Null values
# Check for nulls
# print("Null values before imputation:\n", df_selected.isnull().sum())

# Impute 'Education', 'Occupation', 'Gender', 'MaritalStatus', 'CountryRegionName' with mode
# Impute numerical columns 'Age', 'YearlyIncome', 'AvgMonthSpend', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren' with median
categorical_cols_to_impute = ['Education', 'Occupation', 'Gender', 'MaritalStatus', 'CountryRegionName']
numerical_cols_to_impute = ['Age', 'YearlyIncome', 'AvgMonthSpend', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'HomeOwnerFlag'] # HomeOwnerFlag is also discrete

for col in categorical_cols_to_impute:
    if df_selected[col].isnull().any():
        mode_val = df_selected[col].mode()[0]
        df_selected[col].fillna(mode_val, inplace=True)

for col in numerical_cols_to_impute:
    if df_selected[col].isnull().any():
        median_val = df_selected[col].median()
        df_selected[col].fillna(median_val, inplace=True)

# Convert HomeOwnerFlag to int (it's already 0 or 1 but ensure type)
df_selected['HomeOwnerFlag'] = df_selected['HomeOwnerFlag'].astype(int)
df_selected['BikeBuyer'] = df_selected['BikeBuyer'].astype(int)


# (b) Normalization & (d) Standardization/Normalization
# This will be applied after other transformations, typically StandardScaler for continuous,
# and OneHotEncoder for nominals puts them on a scale (0 or 1).
# For now, define the types:

# (c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different values
# Bin 'Age', 'YearlyIncome', 'AvgMonthSpend' into 5 bins
numerical_for_binning = ['Age', 'YearlyIncome', 'AvgMonthSpend']
for col in numerical_for_binning:
    if df_selected[col].dtype == 'object': # Convert to numeric if not already
        df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')
        # Re-impute if new NaNs were created by coercion
        if df_selected[col].isnull().any():
            df_selected[col].fillna(df_selected[col].median(), inplace=True)

    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform') # or 'quantile'
    # Reshape for KBinsDiscretizer which expects 2D array
    df_selected[f'{col}_Binned'] = discretizer.fit_transform(df_selected[[col]])
    df_selected.drop(col, axis=1, inplace=True)

# (e) Binarization (One Hot Encoding) for nominal attributes
nominal_cols = ['Occupation', 'Gender', 'MaritalStatus', 'CountryRegionName']
ordinal_cols = ['Education'] # Education is ordinal, will use OrdinalEncoder

# Define ordinal categories for Education
education_order = ['Partial High School', 'High School', 'Partial College', 'Bachelors', 'Graduate Degree']
encoder_education = OrdinalEncoder(categories=[education_order], handle_unknown='use_encoded_value', unknown_value=-1)
df_selected['Education_Encoded'] = encoder_education.fit_transform(df_selected[['Education']])
df_selected.drop('Education', axis=1, inplace=True)

# One-Hot Encode nominal features
encoder_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_features = encoder_ohe.fit_transform(df_selected[nominal_cols])
ohe_feature_names = encoder_ohe.get_feature_names_out(nominal_cols)
df_ohe = pd.DataFrame(ohe_features, columns=ohe_feature_names, index=df_selected.index)
df_selected = pd.concat([df_selected.drop(nominal_cols, axis=1), df_ohe], axis=1)

# (d) Standardization/Normalization for numerical features
# After binning and one-hot encoding, apply StandardScaler to remaining numerical columns.
# These include 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'HomeOwnerFlag',
# 'Education_Encoded', and the newly binned columns.
# The target 'BikeBuyer' is binary and should not be scaled for classification.

numerical_for_scaling = [col for col in df_selected.columns if col not in ['BikeBuyer'] and df_selected[col].dtype in ['int64', 'float64']]

# Exclude one-hot encoded columns from standard scaling if they are already 0/1.
# The prompt says "Make sure each attribute is transformed in a same scale for numeric attributes and
# Binarization for each nominal attribute, and each discretized numeric attribute to standardization."
# This implies that binarized (OHE) attributes are fine as they are (0 or 1),
# and numerical/discretized ones should be standardized.

final_numerical_cols = ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
                        'HomeOwnerFlag', 'Education_Encoded',
                        'Age_Binned', 'YearlyIncome_Binned', 'AvgMonthSpend_Binned']

scaler = StandardScaler()
df_selected[final_numerical_cols] = scaler.fit_transform(df_selected[final_numerical_cols])

# Final preprocessed DataFrame
df_preprocessed = df_selected.copy()
# print("\nPreprocessed DataFrame head:\n", df_preprocessed.head())
# print("\nPreprocessed DataFrame info:\n")
# df_preprocessed.info()

# --- Part III: Calculating Proximity/Correlation Analysis of two features ---

# (a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity
# between two objects of your transformed input data.

# Select two arbitrary objects (rows) from the preprocessed data.
object1 = df_preprocessed.iloc[0]
object2 = df_preprocessed.iloc[1]

# Separate binary/nominal (one-hot encoded) features and numerical (scaled) features
# for similarity calculations.
# One-hot encoded features are those that start with the nominal column prefix + '_'
# and the HomeOwnerFlag (which is binary).
binary_nominal_features = [col for col in df_preprocessed.columns if col.startswith(('Occupation_', 'Gender_', 'MaritalStatus_', 'CountryRegionName_')) or col == 'HomeOwnerFlag']
numerical_features_for_cosine = [col for col in df_preprocessed.columns if col not in binary_nominal_features and col != 'BikeBuyer']

obj1_binary_nominal = object1[binary_nominal_features].values
obj2_binary_nominal = object2[binary_nominal_features].values

obj1_numerical_cosine = object1[numerical_features_for_cosine].values
obj2_numerical_cosine = object2[numerical_features_for_cosine].values

# Simple Matching Coefficient (SMC)
# SMC = (number of matching attributes) / (total of attributes)
# Manual implementation for SMC
matching_attributes = np.sum(obj1_binary_nominal == obj2_binary_nominal)
total_attributes = len(obj1_binary_nominal)
smc = matching_attributes / total_attributes if total_attributes > 0 else 0

print(f"\n--- Similarity Calculations for Object 1 and Object 2 ---")
print(f"Simple Matching Coefficient (SMC): {smc:.4f}")


# Ensure inputs are 1D arrays for jaccard_score if they're not already
obj1_binary_nominal_1d = np.squeeze(obj1_binary_nominal)
obj2_binary_nominal_1d = np.squeeze(obj2_binary_nominal)

# Manual calculation of Jaccard Similarity
# M11: number of (1,1) matches
m11 = np.sum((obj1_binary_nominal_1d == 1) & (obj2_binary_nominal_1d == 1))
# M10: number of (1,0) mismatches (obj1 is 1, obj2 is 0)
m10 = np.sum((obj1_binary_nominal_1d == 1) & (obj2_binary_nominal_1d == 0))
# M01: number of (0,1) mismatches (obj1 is 0, obj2 is 1)
m01 = np.sum((obj1_binary_nominal_1d == 0) & (obj2_binary_nominal_1d == 1))

# Jaccard = M11 / (M10 + M01 + M11)
denominator = m10 + m01 + m11
jaccard = m11 / denominator if denominator != 0 else 0 # Handle division by zero

print(f"Jaccard Similarity: {jaccard:.4f}")

# Cosine Similarity
# Cosine similarity is for numerical vectors.
cosine_sim = cosine_similarity(obj1_numerical_cosine.reshape(1, -1), obj2_numerical_cosine.reshape(1, -1))[0][0]
print(f"Cosine Similarity: {cosine_sim:.4f}")

# (b) Calculate Correlation between two features Commute Distance and Yearly Income
# As 'Commute Distance' is not directly present, we use 'NumberCarsOwned' as a proxy.
# We will use the original columns for correlation before extensive one-hot encoding/binning for direct interpretation.
# Load original data again to get un-transformed numerical columns
df_original_numeric = df_merged[['NumberCarsOwned', 'YearlyIncome']].copy()

# Handle potential nulls in these specific columns before correlation if any exist
for col in ['NumberCarsOwned', 'YearlyIncome']:
    if df_original_numeric[col].isnull().any():
        df_original_numeric[col].fillna(df_original_numeric[col].median(), inplace=True)

# Calculate Pearson correlation coefficient
correlation, _ = pearsonr(df_original_numeric['NumberCarsOwned'], df_original_numeric['YearlyIncome'])
print(f"\n--- Correlation Analysis ---")
print(f"Pearson Correlation between 'NumberCarsOwned' (proxy for Commute Distance) and 'YearlyIncome': {correlation:.4f}")


--- Similarity Calculations for Object 1 and Object 2 ---
Simple Matching Coefficient (SMC): 0.8750
Jaccard Similarity: 0.6000
Cosine Similarity: -0.1295

--- Correlation Analysis ---
Pearson Correlation between 'NumberCarsOwned' (proxy for Commute Distance) and 'YearlyIncome': 0.4773
